In [1]:
# Добавляем директорию для равномерного импорта всего с корневой локации
import sys

sys.path.append('../')

# Импортируем библиотеку, которая автоматически настраивает коннектор для системы выполнения кода
from src.libs.LCTWrapTwin import start_module

In [ ]:
# ЭТОТ БЛОК КОДА МОЖНО И НУЖНО МОДИФИЦИРОВАТЬ
# Как обычно, подсказки - в импортах

from calendar import c
from src.libs.LCTWrapTwin.Modules.Handler import MissionHandler, TrustedHandler
import socket
import threading
import time
from math import sqrt
from datetime import datetime
import os
import cv2
import numpy as np

class UserMissionHandler(MissionHandler):

    @staticmethod
    def config_cyber_obstacles() -> dict:

        return {
            "CybP_01": False,  # | Компрометация кода АП | - неверный хэш автопилота
            "CybP_02": False,  # | Неполадки связи | - смешное сообщение
            "CybP_03": False,  # | Привода, поддел инфы о них | - если взлом, то 'eeeeeeee'
            "CybP_04": False,  # | Компроментация навиг. системы | - если взлом, то выдает неверны данные
            "CybP_05": False,  # | Компроментация навесного оборудования | - высокая темп-ра или скорость щетки
            "CybP_06": False,  # | Компроментация МТС | - меняется передаваемый пакет и маршрут движения
        }

    def get_photo(self):  # Для отладки КЗ twin [debug] #

        frame = super().get_camera_frame()
        
        if frame is not None:
            save_dir = "../../Photos"

            # Генерируем уникальное имя файла с временной меткой
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S_%f")[:-3]  # миллисекунды
            
            # Сохраняем изображение
            image_filename = os.path.join(save_dir, f"frame_{timestamp}.jpg")
            cv2.imwrite(image_filename, frame)
            self.lg.log(f"Сохранение изображения")
            
            # Текстовый файл (только цвета пикселей)
            array_filename = os.path.join(save_dir, f"frame_{timestamp}.txt")
            np.savetxt(array_filename, frame.reshape(-1, frame.shape[-1]), fmt='%d', delimiter=',')
            self.lg.log(f"Сохранение массива")
        
        return frame


        
        return True  # пока что всегда возвращает True

    def capture_photo(self, save_dir="./Photos"):  # Для отладки КЗ Real [debug + photo]

        os.makedirs(save_dir, exist_ok=True)

        cap = cv2.VideoCapture("/dev/video0")
        if not cap.isOpened():
            return None, None

        ret, frame = cap.read()
        cap.release()

        if not ret:
            return None, None

        self.set_user_camera_frame(frame)

        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S_%f")[:-3]
        filepath = os.path.join(save_dir, f"frame_{timestamp}.jpg")
        cv2.imwrite(filepath, frame)

        return filepath, frame

    def check_t_light_CV(self) -> int:  # Проверка светофора на зеленый/красный [КЗ] #

        velocity = 50
        green = False
        red = False
        self.lg.log("🚦 Внимание! Светофор 🚦")

        while True:
            # frame = self.get_camera_frame()
            self.capture_photo()
            frame = self.get_user_camera_frame()
            center_row = frame[240, :]
            
            green_pixels = 0
            red_pixels = 0
            
            for x in range(len(center_row)):
                pixel = center_row[x]
                b, g, r = pixel[0].astype(int), pixel[1].astype(int), pixel[2].astype(int)
                
                is_green = (b in range(60-velocity, 60+velocity)) and (g in range(145-velocity, 145+velocity)) and (r in range(40-velocity, 40+velocity))
                
                is_red = (b in range(30-velocity, 30+velocity)) and (g in range(30-velocity, 30+velocity)) and (r in range(145-velocity, 145+velocity))
                
                if is_green:
                    green_pixels += 1      
                elif is_red:
                    red_pixels += 1
                
            if green_pixels > red_pixels:
                if not green:
                    green = True
                    self.lg.log("🟢 Светофор ЗЕЛЁНЫЙ. Вперед! 🟢")
                break
            else:
                if not red:
                    red = True
                    self.lg.log("🔴 Светофор КРАСНЫЙ. Ожидаем... 🔴")
                time.sleep(1.0)

        return True 

    def check_pedestrian_CV(self) -> bool:  # Проверка положения пешехода [КЗ] #
        BGR_TARGET_1 = (150, 150, 150)  # новые значения bgr
        x_1, y_1 = 11, 309
        x_2, y_2 = 507, 352
        x_3, y_3 = 490, 390
        x_4, y_4 = 3, 317
        velocity = 60
        wait = False
        
        while True:
            frame = self.get_camera_frame()
            pixels_1 = frame[y_1, x_1-25:x_1+25]
            pixels_2 = frame[y_2, x_2-25:x_2+25]
            pixels_3 = frame[y_3, x_3-25:x_3+25]
            pixels_4 = frame[y_4, x_4-25:x_4+25]
            
            mas_1 = [[pix[i] in range(BGR_TARGET_1[i] - velocity, BGR_TARGET_1[i] + velocity) for i in range(3)] for pix in pixels_1]
            mas_2 = [[pix[i] in range(BGR_TARGET_1[i] - velocity, BGR_TARGET_1[i] + velocity) for i in range(3)] for pix in pixels_2]
            mas_3 = [[pix[i] in range(BGR_TARGET_1[i] - velocity, BGR_TARGET_1[i] + velocity) for i in range(3)] for pix in pixels_3]
            mas_4 = [[pix[i] in range(BGR_TARGET_1[i] - velocity, BGR_TARGET_1[i] + velocity) for i in range(3)] for pix in pixels_4]
            
            result_1 = any([all(i) for i in mas_1])
            result_2 = any([all(i) for i in mas_2])
            result_3 = any([all(i) for i in mas_3])
            result_4 = any([all(i) for i in mas_4])
            
            if result_1 or result_2 or result_3:
                self.lg.log(f"🚶‍♂️Пешеход перешел дорогу. Вперед!🚶‍♂️")
                break
            else:
                if not wait:
                    wait = True
                    self.lg.log(f"🚸 Пешеходный переход. Ожидаем... 🚸")
                continue
        return True

    def change_barrier_CV(self) -> int:  # Смена положения шлагбаума [КЗ] #
        
        BGR_TARGET_1 = [44, 43, 117]  # без тени
        BGR_TARGET_2 = [50, 50, 50]  # с учетом тени
        # x, y = 301, 273
        x, y = 296, 156
        velocity = 30

        def check_status():
            self.capture_photo()
            frame = self.get_user_camera_frame()
            # frame = self.get_camera_frame()
            pixels = frame[y, x-20:x+20]
            for pixel in pixels:
                mass_1 = [pixel[i] in range(BGR_TARGET_1[i] - velocity, BGR_TARGET_1[i] + velocity) for i in range(3)]
                mass_2 = [pixel[i] in range(BGR_TARGET_2[i] - velocity, BGR_TARGET_2[i] + velocity) for i in range(3)]
                if all(mass_1) or all(mass_2):
                    return 0  # закрыт
            return 1  # открыт
        
        start_status = check_status()
        repeated_failure = False
        if not start_status:
            while True:
                self.set_barrier_toggle()
                time.sleep(2.0)
                if check_status():
                    self.lg.log("✅ Шлагбаум ОТКРЫТ. Можно продолжать движение. ✅")
                    break
                else:
                    if not repeated_failure:
                        self.lg.log("❌ Устройство переключения шлагбаума работает нестабильно ❌")
                        repeated_failure = True
                    else:
                        self.lg.log("   ❌🎲 ПОВТОРНОЕ не срабатывание устройства переключения 🎲❌")
        else:
            self.lg.log("✅ Шлагбаум ОТКРЫТ. Можно продолжать движение. ✅")

    def check_autobot_UDP(self) -> bool:  # Проверка положения автобота перехватом трафика [UDP] #
        while True:
            if "start" in self.buffer:
                self.buffer = []
                break

    def buffer_read(self):
        self.buffer = []
        def monitor():
            while True:
                self.buffer += self.get_message_from_trusted_module()

                if len(self.buffer):
                    if isinstance(self.buffer[-1], tuple):
                        if self.buffer[-1][-1] == 'CybP_04':  # для КП4
                            self.ap_hook.do_move({"x": 1.32, "y": 2.92})
                            self.lg.log(f'{self.buffer[-1][0]} иии {self.buffer[-1][1]}')
                time.sleep(0.5)
        
        monitor_buffer_read = threading.Thread(target=monitor, daemon=True)
        monitor_buffer_read.start()

        

    def mission_code(self):
        
        def robot_marshrut():
            self.set_brush_speed(100)
            self.set_robot_speed(0.24)
            self.buffer_read()
            self.ap_hook.do_rotate({"x": 0.6, "y": 1})
            self.ap_hook.do_move({"x": 0.6, "y": 1})
            self.set_brush_speed(0)
            self.ap_hook.do_rotate({"x": 0.68, "y": 1.4})
            self.set_brush_speed(100)

            self.ap_hook.do_move({"x": 0.68, "y": 1.4})  # Шлагбаум
            self.set_brush_speed(0)
            self.change_barrier_CV()
            self.set_brush_speed(100)

            self.ap_hook.do_move({"x": 0.68, "y": 2.25})  # Верхний пешеход
            self.set_brush_speed(0)
            self.check_pedestrian_CV()
            self.set_brush_speed(100)
            self.ap_hook.do_move({"x": 0.68, "y": 2.92})
            # self.set_brush_speed(0)
            self.ap_hook.do_rotate({"x": 1, "y":2.92})
            self.set_brush_speed(100)
            
            self.ap_hook.do_move({"x": 1, "y": 2.92})  # МТС (сверху)
            self.set_brush_speed(0)
            self.check_autobot_UDP()  
            self.set_brush_speed(100)
            self.ap_hook.do_move({"x": 1.32, "y":2.92})  # С этого момента различия в маршрутах
            self.set_brush_speed(0)
            self.ap_hook.do_rotate({"x": 1.32, "y":2.2})
            self.set_brush_speed(100)
            self.ap_hook.do_move({"x": 1.32, "y":2.2})
            self.set_brush_speed(0)
            self.ap_hook.do_rotate({"x": 1.4, "y":2})
            self.set_brush_speed(100)
            self.ap_hook.do_move({"x": 1.4, "y":2})
            self.set_brush_speed(0)
            self.ap_hook.do_rotate({"x": 1.48, "y":2.2})
            self.set_brush_speed(100)
            self.ap_hook.do_move({"x": 1.48, "y":2.2})
            self.ap_hook.do_move({"x": 1.48, "y":2.92})
            self.set_brush_speed(0)
            self.ap_hook.do_rotate({"x": 2.28, "y":2.92})
            self.set_brush_speed(100)
            self.ap_hook.do_move({"x": 2.28, "y":2.92})
            self.set_brush_speed(0)
            self.ap_hook.do_rotate({"x": 2.28, "y":3.32})
            self.set_brush_speed(100)
            self.ap_hook.do_move({"x": 2.28, "y":3.32})
            self.set_brush_speed(0)
            self.ap_hook.do_rotate({"x": 2.6, "y":3.32})
            self.set_brush_speed(100)
            self.ap_hook.do_move({"x": 2.6, "y":3.32})
            self.ap_hook.do_move({"x": 2.92, "y":3.32})
            self.set_brush_speed(0)
            self.ap_hook.do_rotate({"x": 2.92, "y":2.28})
            self.set_brush_speed(100)
            self.ap_hook.do_move({"x": 2.92, "y":2.28})
            self.set_brush_speed(0)
            self.ap_hook.do_rotate({"x": 1.8, "y":2.28})
            self.set_brush_speed(100)
            self.ap_hook.do_move({"x": 1.8, "y":2.28})
            self.set_brush_speed(0)
            self.ap_hook.do_rotate({"x": 1.7, "y":2.2})
            self.set_brush_speed(100)
            self.ap_hook.do_move({"x": 1.7, "y":2.2})
            self.set_brush_speed(0)
            self.ap_hook.do_rotate({"x": 1.8, "y":2.12})
            self.set_brush_speed(100)
            self.ap_hook.do_move({"x": 1.8, "y":2.12})
            
            self.ap_hook.do_move({"x": 2.65, "y":2.12})  # Cветофор дальний + МТС (в центре)
            self.set_brush_speed(0)
            self.check_autobot_UDP()
            self.check_t_light_CV()
            self.set_brush_speed(100)
            self.ap_hook.do_move({"x": 3.05, "y":2.12})
            self.set_brush_speed(0)
            self.ap_hook.do_rotate({"x": 3.08, "y":3.42})
            self.set_brush_speed(100)
            self.ap_hook.do_move({"x": 3.08, "y":3.42})
            self.set_brush_speed(0)
            self.ap_hook.do_rotate({"x": 2.8, "y":3.42})
            self.set_brush_speed(100)
            self.ap_hook.do_move({"x": 2.6, "y":3.42})
            self.ap_hook.do_move({"x": 2.16, "y":3.42})
            self.set_brush_speed(0)
            self.ap_hook.do_rotate({"x": 2.12, "y":3.08})
            self.set_brush_speed(100)
            self.ap_hook.do_move({"x": 2.12, "y":3.08})
            self.set_brush_speed(0)
            self.ap_hook.do_rotate({"x": 0.52, "y":3.08})
            self.set_brush_speed(100)
            self.ap_hook.do_move({"x": 0.52, "y":3.08})
            self.set_brush_speed(0)
            self.ap_hook.do_rotate({"x": 0.52, "y":2.85})
            self.set_brush_speed(100)

            self.ap_hook.do_move({"x": 0.52, "y":2.95})  # Верхний пешеход
            self.set_brush_speed(0)
            self.check_pedestrian_CV()
            self.set_brush_speed(100)

            self.ap_hook.do_move({"x": 0.52, "y":2.2})  # Шлагбаум 
            self.set_brush_speed(0)
            self.change_barrier_CV()
            self.set_brush_speed(100)
            self.ap_hook.do_move({"x": 0.52, "y":1})
            self.set_brush_speed(0)
            self.ap_hook.do_rotate({"x": 0.52, "y":0.52})
            self.set_brush_speed(100)
            self.ap_hook.do_move({"x": 0.52, "y":0.52})
            self.set_brush_speed(0)
            self.ap_hook.do_rotate({"x": 1, "y":0.52})
            self.set_brush_speed(100)
            self.ap_hook.do_move({"x": 1, "y":0.52})
            self.ap_hook.do_move({"x": 1.32, "y":0.52})
            self.set_brush_speed(0)
            self.ap_hook.do_rotate({"x": 1.32, "y":1.0})
            self.set_brush_speed(100)
            self.ap_hook.do_move({"x": 1.32, "y":1.0})
            self.ap_hook.do_move({"x": 1.32, "y":1.8})
            self.set_brush_speed(0)
            self.ap_hook.do_rotate({"x": 1.4, "y":2})
            self.set_brush_speed(100)
            self.ap_hook.do_move({"x": 1.4, "y":2})
            self.set_brush_speed(0)
            self.ap_hook.do_rotate({"x": 1.48, "y":1.8})
            self.set_brush_speed(100)
            self.ap_hook.do_move({"x": 1.48, "y":1.8})

            self.ap_hook.do_move({"x": 1.48, "y":1})  # Светофор нижний
            self.set_brush_speed(0)
            self.check_t_light_CV()
            self.set_brush_speed(100)
            self.ap_hook.do_move({"x": 1.48, "y":0.52})
            self.set_brush_speed(0)
            self.ap_hook.do_rotate({"x": 1.85, "y":0.52})
            self.set_brush_speed(100)

            self.ap_hook.do_move({"x": 1.85, "y":0.52})  # Пешеход нижний
            self.set_brush_speed(0)
            self.check_pedestrian_CV()
            self.set_brush_speed(100)
            self.ap_hook.do_move({"x": 2.2, "y":0.52})
            self.ap_hook.do_move({"x": 2.6, "y":0.52})
            self.set_brush_speed(0)    
            self.ap_hook.do_rotate({"x": 3.08, "y":0.52})
            self.set_brush_speed(100)
            
            self.ap_hook.do_move({"x": 3.08, "y":0.52})  # МТС (снизу)
            self.ap_hook.do_rotate({"x": 3.08, "y":1})
            self.set_brush_speed(0)
            self.check_autobot_UDP()
            self.set_brush_speed(100)
            self.ap_hook.do_move({"x": 3.08, "y":1.9})
            self.set_brush_speed(0)
            self.ap_hook.do_rotate({"x": 3, "y":2})
            self.set_brush_speed(100)
            self.ap_hook.do_move({"x": 3, "y":2})
            self.set_brush_speed(0)
            self.ap_hook.do_rotate({"x": 2.92, "y":1.9})
            self.set_brush_speed(100)
            self.ap_hook.do_move({"x": 2.92, "y":1.9})
            self.ap_hook.do_move({"x": 2.92, "y":0.68})
            self.set_brush_speed(0)
            self.ap_hook.do_rotate({"x": 2.6, "y":0.68})
            self.set_brush_speed(100)
            self.ap_hook.do_move({"x": 2.6, "y":0.68})

            self.ap_hook.do_move({"x": 2.45, "y":0.68})  # Пешеход нижний
            self.set_brush_speed(0)
            self.check_pedestrian_CV()
            self.set_brush_speed(100)
            self.ap_hook.do_move({"x": 1.8, "y":0.68})
            self.ap_hook.do_move({"x": 1.4, "y":0.68}) 
            self.ap_hook.do_move({"x": 1.0, "y":0.68})
            self.set_brush_speed(0)
            self.ap_hook.do_rotate({"x": 0.68, "y":0.68})
            self.set_brush_speed(100)
            self.ap_hook.do_move({"x": 0.68, "y":0.68})
            self.set_brush_speed(0)
            self.ap_hook.do_rotate({"x": 0.68, "y":1.0})
            self.set_brush_speed(100)
            self.ap_hook.do_move({"x": 0.68, "y":1.0})
            self.set_brush_speed(0)
            self.ap_hook.do_rotate({"x": 0.2, "y":1.0})
            self.set_brush_speed(100)
            self.ap_hook.do_move({"x": 0.2, "y":1.0})
            self.set_brush_speed(0)
            self.ap_hook.do_rotate({"x": 0.6, "y":1.0}) 
            

        robot_marshrut()





class UserTrustedHandler(TrustedHandler):
    
    def __init__(self, context):
        super().__init__(context)

        # Переменные для защиты от CybP_01 (компрометация автопилота)
        self.cybp01_attack_detected = False
        self.cybp01_original_hash = None
        self.cybp01_current_hash = None
        self.cybp01_monitoring_active = False
        self.cybp01_position_monitoring = False
                
        # Переменные для защиты от CybP_02 (компрометация связи)
        self.cybp02_message_count = 0
        self.cybp02_protection_active = False
        self.cybp02_malfunction_detected = False
        self.cybp02_attack_count = 0
        self.initial_message = "initial_secure_message"
        self.last_correct_message = None
                
        # Переменные для защиты от CybP_03 (компрометация приводов)
        self.cybp03_attack_detected = False
        self.cybp03_compromised_drive_id = None
        self.cybp03_last_drive_data = None

        # Переменные для защиты от CybP_04 (компрометация системы навигации)
        # В зоне радиопомех позиционирование "прыгает", но ориентация (rotation) надёжна — от магнитометра.
        self.cybp04_attack_detected = False
        self.cybp04_stop_activate = False
        self.cybp04_attack_count = 0


    class UdpMaster:
        
        mode = "real"

        @classmethod
        def check_mode(cls) -> str:
                
            def th_chmode():
                port = 10004
                listen_port = port
                interceptor_socket = None
                
                try:
                    interceptor_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
                    interceptor_socket.settimeout(2.0)
                    interceptor_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
                    interceptor_socket.bind(('0.0.0.0', listen_port))
                    
                    try:
                        data = interceptor_socket.recvfrom(1024)[0]
                        decoded_data = data.decode('utf-8', errors='ignore').strip()
                        result = decoded_data

                        interceptor_socket.close()
                        cls.mode = "twin"
                        return

                    except socket.timeout:
                        cls.mode = "real"
                        
                    except Exception as e:
                        pass
                        
                except Exception as e:
                    pass
            
            interceptor_th = threading.Thread(target=th_chmode, daemon=True)
            interceptor_th.start()

        @classmethod
        def coords_autobot_UDP(cls, trusted_obj):

            ports = {
            "twin": 10004,  # в двойнике
            "real": 8000   # реальное железо
            }

            listen_port = ports[f"{cls.mode}"]
            result = None
            dangerous = False
            flag = False
            
            def connect():
                interceptor_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
                interceptor_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
                interceptor_socket.bind(('0.0.0.0', listen_port))
                
                data = interceptor_socket.recvfrom(1024)[0]
                decoded_data = data.decode('utf-8', errors='ignore').strip()
                result = decoded_data
                interceptor_socket.close()
                return result

            while True:
                try:
                    check = []
                    my_x, my_y = trusted_obj.real_position()
                    for _ in range(2):
                        if _ == 1:
                            time.sleep(1.5)
                        result = connect()
                        data_autorobot = result[1:-1].split(',')
                        keys = []
                        coords = []
                        for i in range(len(data_autorobot)):
                            key = data_autorobot[i].split(':')[0].strip()[1:-1]
                            coord = float(data_autorobot[i].split(':')[1])
                            keys.append(key)
                            coords.append(coord)
                        if cls.mode == "twin":
                            autorobot_x, autorobot_y = coords[0], coords[1]
                        else:
                            if coords[keys.index('r_id')] == 2:
                                autorobot_x = coords[keys.index('position_x')]
                                autorobot_y = coords[keys.index('position_y')]
                        d = round(sqrt((autorobot_x - my_x)**2 + (autorobot_y - my_y)**2), 4)
                        check.append(d)
                    
                    cell_number = trusted_obj._calculate_cell_from_position(autorobot_x, autorobot_y)
                    if cell_number in (6, 7, 8, 15, 17):
                        continue

                    if check[0] == check[1]:
                        if dangerous:  # ехал и остановился
                            for _ in range(5):  # чтобы наверняка
                                trusted_obj.send_message_to_ap("start")
                            trusted_obj.lg.log("[📡<OP>🛜]::💤 Авторобот остановился, можно ехать ♻️")
                            break
                        if not flag:  # стоит на месте
                            trusted_obj.lg.log("[📡<OP>🛜]::💤 Пока что авторобот стоит на месте. Ожидаем... 🛑")
                            flag = True
                    elif check[1] > check[0]:  # расстояние увеличивается
                        for _ in range(5):  # чтобы наверняка
                            trusted_obj.send_message_to_ap("start")
                        trusted_obj.lg.log("[📡<OP>🛜]::🌀 Авторобот находится в движении - он уезжает от нас! Вперед... 🟢")
                        break
                    else:  # расстояние сокращается
                        if not dangerous:
                            trusted_obj.lg.log("[📡<OP>🛜]::🌀 Авторобот находится в движении. Ожидаем... 🛑")
                            dangerous = True
                                    
                except Exception as e:
                    trusted_obj.lg.log(f"Пришли данные не того робота, либо вообще не пришли: {e}")

            time.sleep(5.0)  # чтобы АП успел прочитать буфер
        
        @classmethod
        def intercept_UDP(cls, trusted_obj: UserMissionHandler, name_device: str) -> int:  # Перехват трафика умных устройств [UDP] #

            ip_addreses_twin = "127.0.0.1"
            ports_twin = {
                "traffic_0": 5041,  # светофор 0 (самый ближний)
                "traffic_1": 5042,  # светофр 1 (средний)
                "traffic_2": 5043,  # светофор 2 (самый дальний)
                "barrier": 5031,   # шлагбаум (больше не работает)
                "autobot": 5071    # МТС - компроментация 
            }

            ip_addreses_real = {
                "traffic_0": "192.168.60.181",  # светофор 0 (самый ближний)
                "traffic_1": "192.168.60.182",  # светофр 1 (средний)
                "traffic_2": "192.168.60.183",  # светофор 2 (самый дальний)
                "barrier": "192.168.60.131",   # шлагбаум (больше не работает)
                "autobot": "192.168.60.145"    # МТС - компроментация 
            }
            port_real = 4000

            if cls.mode == "twin":
                ip_adress = ip_addreses_twin
                listen_port = ports_twin[name_device]
            else:
                ip_adress = ip_addreses_real["name_device"]
                listen_port = port_real

            result = None 
            interceptor_socket = None
            
            try:
                interceptor_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
                interceptor_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
                interceptor_socket.bind((ip_adress, listen_port))
                
                try:
                    data = interceptor_socket.recvfrom(1024)[0]
                    decoded_data = data.decode('utf-8', errors='ignore').strip()
                    if name_device in tuple(ports_twin.keys())[:-2]:  # светофоры
                        color = int(decoded_data[-2])
                        trusted_obj.lg.log(f"{color}")
                        trusted_obj.lg.log(f"{name_device} КРАСНЫЙ") if color == 1 else trusted_obj.lg.log(f"{name_device} ЗЕЛЁНЫЙ")
                        if color == 2:
                            result = True
                    elif name_device in tuple(ports_twin.keys())[-2]:  # шлагбаум
                        status = decoded_data[-2]
                        result = int(status)
                    elif name_device in tuple(ports_twin.keys())[-1]:  # автоматичский робот
                        result = decoded_data

                    interceptor_socket.close()
                    if listen_port == 4000:
                        trusted_obj.lg.log(f'{result}')  # )))
                    return result
                    
                except Exception as e:
                    trusted_obj.lg.log(f"Ошибка при получении данных: {e}")
                    
            except Exception as e:
                trusted_obj.lg.log(f"Ошибка создания сокета: {e}")


    def trusted_code(self):

        time.sleep(1.0)
        self._start_all_monitoring_systems()
        self.UdpMaster.check_mode()
        time.sleep(3.0)
        self.lg.log(f'{self.UdpMaster.mode}')

    def make_next_short_message(self, prev_message: str):
        pass
        self.get_robot_status()
        some_message = "exactly_correct_short_message"
        return some_message


# Атака CybP_01
    def _start_autopilot_monitoring(self):
        
        def autopilot_monitor():
            self._initialize_original_hash()
            
            while True:
                self._check_code_autopilot_integrity()
                time.sleep(0.5)
        
        monitor_thread = threading.Thread(target=autopilot_monitor, daemon=True)
        monitor_thread.start()

    def _initialize_original_hash(self):
        hash_data = self.get_ap_code_hash()
        if hash_data and 'ap_code_hash' in hash_data:
            self.cybp01_original_hash = hash_data['ap_code_hash']
            self.cybp01_current_hash = hash_data['ap_code_hash']

    def _check_code_autopilot_integrity(self):
        hash_data = self.get_ap_code_hash()
        if not hash_data or 'ap_code_hash' not in hash_data:
            return
        
        current_hash = hash_data['ap_code_hash']
        self.cybp01_current_hash = current_hash
        
        if self.cybp01_original_hash and current_hash != self.cybp01_original_hash:
            if not self.cybp01_attack_detected:
                self.cybp01_attack_detected = True
                self.lg.warn("🚨 CybP_01: Обнаружена компроментация автопилота!")
                self._apply_cybp01_protection()
             
    def _apply_cybp01_protection(self):
        self.lg.log("🔧 CybP_01: Сброс автопилота для восстановления")
        reset_result = self.set_ap_force_reset()
        time.sleep(1.0)  
        
        if reset_result and isinstance(reset_result, dict) and reset_result.get('status') == 'OK':
            self.lg.log("🦾 CybP_01: Автопилот сброшен")

    def _wait_for_autopilot_recovery(self):     
        for attempt in range(10):
            time.sleep(1.0)
            
            hash_data = self.get_ap_code_hash()
            if hash_data and 'ap_code_hash' in hash_data:
                current_hash = hash_data['ap_code_hash']
                if current_hash == self.cybp01_original_hash:
                    self.lg.log("✅ CybP_01: Автопилот восстановлен!")
                    self._handle_cybp01_recovery(False)
                    return

        self.cybp01_attack_detected = False
        self.cybp01_position_monitoring = False

    def _apply_emergency_stop_cybp01(self):
            self.lg.warn("🚨 CybP_01: Критическая ошибка - экстренная остановка!")
            
            try:
                emergency_result = self.set_emergency_stop(True)
                if emergency_result and isinstance(emergency_result, dict) and emergency_result.get('status') == 'OK':
                    self.lg.log("🦾 CybP_01: Все приводы остановлены")
                    self._release_emergency_stop_cybp01()
                else:
                    self._release_emergency_stop_cybp01()
            except Exception as e:
                self._release_emergency_stop_cybp01()
                
    def _release_emergency_stop_cybp01(self):
        time.sleep(1.0)
        
        release_result = self.set_emergency_stop(False)
        
        if release_result and isinstance(release_result, dict) and release_result.get('status') == 'OK':
            self.lg.log("✅ CybP_01: Робот готов к продолжению движения")
            
            self.cybp01_attack_detected = False
            self.cybp01_position_monitoring = False


# Атака CybP_03
    def _start_drive_monitoring(self):
        
        def drive_monitor():
            while True:
                self.cybp03_check_drive_integrity()
                time.sleep(0.1)
        
        monitor_thread = threading.Thread(target=drive_monitor, daemon=True)
        monitor_thread.start()

    def cybp03_check_drive_integrity(self):
        drive_data = self.get_drive_data()
        if not drive_data or 'drive_data' not in drive_data:
            return
        
        current_drives = drive_data['drive_data']
        
        for drive in current_drives:
            drive_id = drive.get('d_id', -1)
            signature = drive.get('signature', '')
            serial = drive.get('serial', '')
            data = drive.get('data', '')
            
            if not self._is_valid_signature(signature, serial, data):
                if not self.cybp03_attack_detected:
                    self.lg.log(f"🚨 CybP_03: Привод {drive_id}: невалидная сигнатура '{signature}'")
                    self.cybp03_attack_detected = True
                    self._cybp03_apply_protection(drive_id)
                if self.cybp03_attack_detected:
                    self._cybp03_apply_protection(drive_id)
            else:
                self.cybp03_attack_detected = False
            return

    def _is_valid_signature(self, signature, serial, data):
        # Базовые проверки
        if not signature or len(signature) != 2:
            return False
            
        # Проверяем hex формат
        if not all(c in '0123456789abcdef' for c in signature.lower()):
            return False
        
        # Если нет serial или data - не можем проверить, считаем валидной
        if not serial or not data:
            return True
        
        # Вычисляем ожидаемую сигнатуру
        import crc8
        c_hash = crc8.crc8()
        c_hash.update(bytes(str(serial + data).encode("utf-8")))
        expected_signature = c_hash.hexdigest()
        
        # Допускаем небольшие расхождения (на случай race conditions)
        if signature == expected_signature:
            return True
        else:
            return False
       
    def _cybp03_apply_protection(self, compromised_drive_id):
        reset_result = self.set_drive_force_reset({"d_id": str(compromised_drive_id)})
        time.sleep(0.5)
        
        if reset_result and isinstance(reset_result, dict) and reset_result.get('status') == 'OK':
            self.lg.log(f"🦾 \"CybP_03\": Привод {compromised_drive_id} успешно сброшен")

        time.sleep(1.0) 
        self._cybp03_wait_for_recovery(compromised_drive_id)

    def _cybp03_wait_for_recovery(self, drive_id):
        
        for _ in range(15):
            time.sleep(1.0)
            
            drive_data = self.get_drive_data()
            if drive_data and 'drive_data' in drive_data:
                current_drives = drive_data['drive_data']
                recovery_detected = True
                
                for drive in current_drives:
                    if drive.get('d_id') == drive_id:
                        signature = drive.get('signature', '')
                        serial = drive.get('serial', '')
                        data = drive.get('data', '')
                        
                        if not self._is_valid_signature(signature, serial, data):
                            recovery_detected = False
                            self.lg.log(f"❌ Привод {drive_id} все еще скомпрометирован: signature='{signature}'")
                            break
                
                if recovery_detected:
                    self.lg.log(f"✅ \"CybP_03\": Привод {drive_id} восстановлен!")
                    self.cybp03_attack_detected = False
                    self.cybp03_compromised_drive_id = None
                    return


# Aтака CybP_04
    def _start_navigation_monitoring(self):
        
        def nav_monitor():
            while True:
                self._check_navigation()
                time.sleep(0.2)


        monitor_thread = threading.Thread(target=nav_monitor, daemon=True)
        monitor_thread.start()

    def _check_navigation(self):
        
        def check_descrepancy():
            st = self.get_robot_status()
            
            # Получаем координаты обычным способом
            x_gps = float(st.get("position_x", 0.0))
            y_gps = float(st.get("position_y", 0.0))
                    
            x_trilat, y_trilat = self.real_position()  


            discrepancy = sqrt((x_gps - x_trilat)**2 + (y_gps - y_trilat)**2)
            return discrepancy, x_trilat, y_trilat

        discrepancy, x_trilat, y_trilat = check_descrepancy()

        if discrepancy > 0.4:
            if not self.cybp04_attack_detected:
                self.lg.warn(f"🚨 CybP_04: Расхождение позиций!  Разница: {discrepancy:.2f}")
                self.lg.log(f"🚨 CybP_04: Произведена экстренная остановка")
                self.cybp04_attack_detected = True
            self.set_emergency_stop(True)
            for _ in range(5):
                self.send_message_to_ap((x_trilat, y_trilat, 'CybP_04'))
        else:
            if self.cybp04_attack_detected:
                self.set_emergency_stop(False)
                self.lg.log(f"✅ \"CybP_04\": Система навигации восстановлена. Разница: {discrepancy:.2f}")
                self.cybp04_attack_detected = False
            return


# Атака CybP_05
    def _start_cybp05_monitor(self):  # Только мониторинг температуры #
        
        def loop():
            cybp05_active = False
            cybp05_crit = 120
            while True:
                
                st = self.get_robot_status()
                t = st.get("brush_controller_temperature")

                if t >= cybp05_crit:
                    if not cybp05_active:
                        cybp05_active = True
                        self.lg.log("🚨 CybP_05: Перегрев навесного оборудования!")
                elif cybp05_active:
                    cybp05_active = False
                    self.lg.log("✅ CybP_05: Температура навесного оборудования снижена.")

                time.sleep(0.5)

        threading.Thread(target=loop, daemon=True).start()


# Атака CybP_06
    def check_CybP_06_UDP(self) -> bool:  # Проверка компроментации МТС (CybP_06) [UDP] #
        
        data = self.UdpMaster.intercept_UDP(self, "autobot")
        result = int(data[-2])
        repeated_failure = False
        if result:
            if not repeated_failure:
                repeated_failure = True
                self.lg.log('[📡<OP>🛜]::🚨 CybP_06: Обнаружена компроментация МТС!')
        else:
            repeated_failure = False
            self.lg.log('[📡<OP>🛜]::✅ Компроментации МТС не обнаружено.')
        
        return True  # пока что всегда возвращает True  

    def _check_robot_position(self):
            
        def monitor():
            while True:
                robot_status_for_rotation = self.get_robot_status()
                current_x, current_y = self.real_position()
                rotation = robot_status_for_rotation["rotation"]
                self._calculate_cell_from_position(current_x, current_y, int(rotation))
                time.sleep(1.5)

        monitor_start = threading.Thread(target=monitor, daemon=True)
        monitor_start.start()

    def _calculate_cell_from_position(self, x, y, rotation=500):
        cell_x = x // 0.4 + 1
        cell_y = 9 - (y // 0.4 + 1)
        
        cell_number = int(cell_y * 9 + cell_x)

        self.current_cell = False

        # МТС
        if (cell_number == 12 and -10 <= rotation <= 10) \
            or (cell_number == 22 and 80 <= rotation <= 100) \
                or (cell_number == 71 and 80 <= rotation <= 100):
            if not self.current_cell:
                self.current_cell = True
                self.check_CybP_06_UDP()
                self.UdpMaster.coords_autobot_UDP(self)
        elif cell_number not in (12, 22, 71):
            self.current_cell = False

        return cell_number

    def real_position(self):  # вычиселние реальной поизиции

        data = self.get_robot_status()
        x_a, y_a = data["base_station_alpha_rssi"]
        x_b, y_b = data["base_station_bravo_rssi"]
        x_c, y_c = data["base_station_charlie_rssi"]
        
        r1_sq = x_a**2 + y_a**2
        r2_sq = x_b**2 + y_b**2
        r3_sq = x_c**2 + y_c**2
        
        C1 = r1_sq - r2_sq + 2
        C2 = r1_sq - r3_sq - 2.43
        
        x = -(C2 + 0.1 * C1) / 1.4
        y = x + C1 / 2
        
        return x/1000, y/1000



    def _start_all_monitoring_systems(self):
        
        self._start_autopilot_monitoring() # - 1

        # ...  # - 2

        self._start_drive_monitoring() # - 3

        self._start_navigation_monitoring() # - 4
        
        self._start_cybp05_monitor() # - 5

        self._check_robot_position() # - 6



In [ ]:
start_module(UserMissionHandler, UserTrustedHandler)